In [4]:
import os
import rioxarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import csv
import math
from osgeo import gdal, ogr, osr, gdalconst


In [1]:
def tif_dict(strpath, masked=True, chunks=None, dsmatch=None):
    """
    Walks a directory of geotiffs and returns a dictionary of rioxarray DataArrays
    Args:
        strpath: directory name
        chunks: tuple of integers
        dsmatch: data array to match the directory of raster to
        masked: whether to mask by nodata, default is yes, pass masked=False if not desired
    Returns:
        a dictionary of rioxarrays
        
    Examples: 
       tif_dict(r'D:\BananaSplits')
       tif_dict(r'D:\BananaSplits', chunk=s(1,1024,1024))
    
    """

    check_dict = {}
    for root, dirs, files in os.walk(strpath):
        for file in files:
            if '.tif' in file and '.xml' not in file:
                if dsmatch is None:
                    if chunks is None:
                        check_dict[file] = rioxarray.open_rasterio(os.path.join(root,file),masked=masked)    
                    else:
                        check_dict[file] = rioxarray.open_rasterio(os.path.join(root,file), masked=masked, chunks=chunks)    
                else:
                    if chunks is None:
                        check_dict[file] = rioxarray.open_rasterio(os.path.join(root,file), masked=masked).rio.reproject_match(dsmatch)    
                    else:
                        check_dict[file] = rioxarray.open_rasterio(os.path.join(root,file), masked=masked, chunks=chunks).rio.reproject_match(dsmatch)    
                
    return check_dict


A good comparison piece would be: https://search.geoscience.nsw.gov.au/product/9233/7564550 in the Curnamona

here's a starting point comparison https://github.com/chudasama-bijal/QGIS-Plugin-Weights-of-Evidence-Model

An R version - https://github.com/GeoscienceAustralia/MPM-WofE

Get training data vector data - 1s

Choose mask layer if desired

Classes
- Favourable
- Unfavourable

Get list of evidence rasters

Studentized t contrast

Clip and make rasters the same details

Rasterise training data

Get count of data and not - e.g 1s and 0s in raster

Calculate weights
#633

Input weights rasters

calculate bayesian probs
#1590

In [5]:
## so possible exercise - convert the R version to python, possibly with robot assistance
## vectorize more in xarray

Borrow the test dataset from the Geoscience Australia example above by Arianne Ford